<a href="https://colab.research.google.com/github/extrajp2014/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/module1-logistic-regression/LS_DS_231_Logistic_Regression_DS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [0]:
!pip install kaggle

In [0]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
401 - Unauthorized


In [0]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [0]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [0]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [0]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [0]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [0]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [0]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [0]:
log_reg.predict(test_case)[0]

1

In [0]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [0]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [0]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [0]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [0]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [0]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip

--2019-02-25 18:18:08--  http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/zip]
Saving to: ‘Absenteeism_at_work_AAA.zip.3’

Absenteeism_at_work 100%[===================>]  64.59K   179KB/s    in 0.4s    

2019-02-25 18:18:08 (179 KB/s) - ‘Absenteeism_at_work_AAA.zip.3’ saved [66136/66136]



In [0]:
!unzip Absenteeism_at_work_AAA.zip

Archive:  Absenteeism_at_work_AAA.zip
replace Absenteeism_at_work.arff? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Absenteeism_at_work.arff  
replace Absenteeism_at_work.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Absenteeism_at_work.csv  
replace Absenteeism_at_work.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Absenteeism_at_work.xls  
replace Attribute Information.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Attribute Information.docx  
replace UCI_ABS_TEXT.docx? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: UCI_ABS_TEXT.docx       


In [0]:
!head Absenteeism_at_work.csv

ID;Reason for absence;Month of absence;Day of the week;Seasons;Transportation expense;Distance from Residence to Work;Service time;Age;Work load Average/day ;Hit target;Disciplinary failure;Education;Son;Social drinker;Social smoker;Pet;Weight;Height;Body mass index;Absenteeism time in hours
11;26;7;3;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;4
36;0;7;3;1;118;13;18;50;239.554;97;1;1;1;1;0;0;98;178;31;0
3;23;7;4;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
7;7;7;5;1;279;5;14;39;239.554;97;0;1;2;1;1;0;68;168;24;4
11;23;7;5;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;2
3;23;7;6;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
10;22;7;6;1;361;52;3;28;239.554;97;0;1;1;1;0;4;80;172;27;8
20;23;7;6;1;260;50;11;36;239.554;97;0;1;4;1;0;0;65;168;23;4
14;19;7;2;1;155;12;14;34;239.554;97;0;1;2;1;0;0;95;196;25;40


In [0]:
pd.set_option('display.max_columns', 50)
df = pd.read_csv('Absenteeism_at_work.csv', sep=';')
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


In [0]:
df.shape

(740, 21)

In [0]:
df['Reason for absence'].value_counts()

23    149
28    112
27     69
13     55
0      43
19     40
22     38
26     33
25     31
11     26
10     25
18     21
14     19
1      16
7      15
6       8
12      8
8       6
21      6
9       4
5       3
24      3
16      3
4       2
15      2
3       1
2       1
17      1
Name: Reason for absence, dtype: int64

In [0]:
df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [0]:
X = df.drop(['ID', 'Reason for absence'], axis=1)

In [0]:
y = df['Reason for absence']

In [0]:
from sklearn.linear_model.logistic import LogisticRegression

# LogisticRegression

In [0]:
# import sklearn
# sklearn.__version__

model = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial', max_iter=10000)
model.fit(X,y)
model.score(X,y)

0.5121621621621621

In [0]:
?LogisticRegression

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=.2, random_state=42)

model2 = LogisticRegression(random_state=42, multi_class='multinomial', solver='lbfgs', max_iter=8000)
model2.fit(X_train,Y_train)
model2.score(X_test,Y_test)

0.3716216216216216

In [0]:
predictions = model2.predict(X_test)
predictions

array([27, 23, 15, 23, 27, 23, 23, 13, 22, 28,  0,  0, 23, 13,  0,  1, 23,
       23, 14, 28, 22, 23, 13,  0, 23, 28, 28, 27, 23, 13, 24, 23, 24, 28,
       23, 28, 23, 23, 13, 19, 22, 22, 22, 28, 23, 27, 26, 19, 23, 23, 27,
       18, 23, 28, 25,  0, 28,  0, 21, 19, 11, 28, 13,  7, 26, 19, 23, 28,
       27, 13, 27, 22, 13, 16, 23, 10, 23, 27, 10, 10, 28, 23, 13, 22, 22,
       27, 28, 18, 22, 10, 28,  0, 23, 28, 23, 27, 23, 10, 27, 11, 23,  7,
       14, 23, 27, 27, 22, 27,  0, 27, 26, 25, 28, 28, 13, 10, 27, 23, 14,
       19, 23, 23, 26, 28, 27, 23,  0, 23, 22, 23, 27, 23, 27, 28,  5, 18,
       26, 23, 23, 23, 23, 28, 19, 23, 27, 27, 23, 23])

In [0]:
from collections import Counter
Counter(predictions)

Counter({0: 9,
         1: 1,
         5: 1,
         7: 2,
         10: 6,
         11: 2,
         13: 10,
         14: 3,
         15: 1,
         16: 1,
         18: 3,
         19: 6,
         21: 1,
         22: 11,
         23: 41,
         24: 2,
         25: 2,
         26: 5,
         27: 21,
         28: 20})

In [0]:
# import numpy as np
single_row = np.array(X_test.iloc[0])
single_row

array([  1.   ,   6.   ,   2.   , 225.   ,  26.   ,   9.   ,  28.   ,
       308.593,  95.   ,   0.   ,   1.   ,   1.   ,   0.   ,   0.   ,
         2.   ,  69.   , 169.   ,  24.   ,   1.   ])

In [0]:
model2.predict([single_row])[0]

27

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [0]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-02-26 06:02:34--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  17.8MB/s    in 20s     

2019-02-26 06:02:55 (16.9 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [0]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
# !pip install --upgrade seaborn
import seaborn as sns
sns.__version__
from sklearn.linear_model.logistic import LogisticRegression
import statsmodels.api as sm

pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
pd.options.display.float_format = '{:,}'.format

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
# Clean data
df = pd.read_csv('fma_metadata/tracks.csv', header=[1])
df = df.drop(['Unnamed: 0'], axis=1) # drop track column
df= df.drop([0, 1]) # drop a whole row of nan

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# preview data
print("df shape:"), print(df.shape), print("---"*20)
print("df columns:"), print(df.columns), print("---"*20)
print("df select_dtypes(include=[np.number]).columns.values:"), print(df.select_dtypes(include=[np.number]).columns.values), print("---"*20)
print("df select_dtypes(exclude=[np.number]).columns:"), print(df.select_dtypes(exclude=[np.number]).columns), print("---"*20)
print("df dtypes.sort_values(ascending=False):"), print(df.dtypes.sort_values(ascending=False)), print("---"*20)
print("df head().T:"), print(df.head().T), print("---"*20)
print("df isnull().sum().sum():"), print(df.isnull().sum().sum()), print("---"*20)
print("df isna().sum().sort_values(ascending=False):"), print(df.isna().sum().sort_values(ascending=False)), print("---"*20)
# nan finder
print("columns[df.isna().any()].tolist():"), print(df.columns[df.isna().any()].tolist()), print("")
# stats data
print("df corr().T:"), print(df.corr().T), print("")
print("df describe(include='all').T:"), print(df.describe(include='all').T), print("")

df shape:
(106573, 52)
------------------------------------------------------------
df columns:
Index(['comments', 'date_created', 'date_released', 'engineer', 'favorites', 'id', 'information', 'listens', 'producer', 'tags', 'title', 'tracks', 'type', 'active_year_begin', 'active_year_end', 'associated_labels', 'bio', 'comments.1', 'date_created.1', 'favorites.1', 'id.1', 'latitude', 'location', 'longitude', 'members', 'name', 'related_projects', 'tags.1', 'website', 'wikipedia_page', 'split', 'subset', 'bit_rate', 'comments.2', 'composer', 'date_created.2', 'date_recorded', 'duration',
       'favorites.2', 'genre_top', 'genres', 'genres_all', 'information.1', 'interest', 'language_code', 'license', 'listens.1', 'lyricist', 'number', 'publisher', 'tags.2', 'title.1'],
      dtype='object')
------------------------------------------------------------
df select_dtypes(include=[np.number]).columns.values:
['comments' 'favorites' 'id' 'listens' 'tracks' 'comments.1' 'favorites.1'
 'id.1' 

(None, None, None)

In [0]:
# More cleaning
# list of columns with nan over 20,000
df.isna().sum().sort_values(ascending=False).index
nanList=['lyricist', 'publisher', 'information.1', 'composer', 'active_year_end', 
         'wikipedia_page', 'date_recorded', 'related_projects', 'associated_labels', 
         'language_code', 'engineer', 'producer', 'active_year_begin', 'latitude', 
         'longitude', 'members', 'location', 'date_released', 'bio', 
         'website', 'information']
df= df.drop(nanList, axis=1)
df= df.dropna()
df.reset_index(drop=True, inplace=True)

In [0]:

print(df.genre_top.unique())
print(len(df.genre_top.unique()))

['Hip-Hop' 'Pop' 'Rock' 'Experimental' 'Folk' 'Jazz' 'Electronic' 'Spoken'
 'International' 'Soul-RnB' 'Blues' 'Country' 'Classical'
 'Old-Time / Historic' 'Instrumental' 'Easy Listening']
16


In [0]:
# Setup data source for regression
X = df._get_numeric_data() # only choose columns with numeric values
y = df[['genre_top']].dropna().values
print(X.columns.values)
print(X.head(10).T)
print(X.shape)
print(y.shape)

['comments' 'favorites' 'id' 'listens' 'tracks' 'comments.1' 'favorites.1'
 'id.1' 'bit_rate' 'comments.2' 'duration' 'favorites.2' 'interest'
 'listens.1' 'number']
                    0         1         2         3         4         5         6         7         8         9
comments          0.0       0.0       0.0       0.0       1.0       1.0       1.0       1.0       0.0       1.0
favorites         4.0       4.0       4.0       4.0       0.0       0.0       2.0       2.0       1.0       1.0
id                1.0       1.0       6.0       1.0      58.0      58.0      59.0      59.0      60.0      61.0
listens       6,073.0   6,073.0  47,632.0   6,073.0   3,331.0   3,331.0   1,681.0   1,681.0   1,304.0   1,300.0
tracks            7.0       7.0       2.0       7.0       4.0       4.0       2.0       2.0       2.0       1.0
comments.1        0.0       0.0       3.0       0.0       1.0       1.0       0.0       0.0       0.0       0.0
favorites.1       9.0       9.0      74.0       9.

In [0]:
# test 1 - no training
from sklearn.linear_model import LogisticRegression
import numpy as np

model = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial', max_iter=1000)
model.fit(X,y)
model.score(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.39104622665427197

In [0]:
# test 2 - with training
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs', multi_class = 'multinomial', max_iter=1000)
model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/

0.3852362038367278

In [0]:
X = df[['favorites','favorites.1','favorites.2','comments','comments.1','comments.2','number','tracks','duration']]
y = df[['genre_top']].dropna().values
print(X.columns.values)
print(X.head(10).T)
print(X.shape)
print(y.shape)

['favorites' 'favorites.1' 'favorites.2' 'comments' 'comments.1'
 'comments.2' 'number' 'tracks' 'duration']
                0     1     2     3     4     5       6       7     8     9
favorites     4.0   4.0   4.0   4.0   0.0   0.0     2.0     2.0   1.0   1.0
favorites.1   9.0   9.0  74.0   9.0   0.0   0.0     5.0     5.0  11.0  11.0
favorites.2   1.0   6.0 178.0   3.0   0.0   0.0     2.0     2.0   3.0   5.0
comments      0.0   0.0   0.0   0.0   1.0   1.0     1.0     1.0   0.0   1.0
comments.1    0.0   0.0   3.0   0.0   1.0   1.0     0.0     0.0   0.0   0.0
comments.2    0.0   0.0   0.0   0.0   1.0   1.0     0.0     0.0   0.0   0.0
number        4.0   6.0   1.0   5.0   0.0   0.0     1.0     2.0   2.0   2.0
tracks        7.0   7.0   2.0   7.0   4.0   4.0     2.0     2.0   2.0   1.0
duration    237.0 206.0 161.0 207.0 837.0 509.0 1,233.0 1,231.0 296.0 253.0
(47332, 9)
(47332, 1)


In [0]:
# test 3 
# mod: new dataset with training
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs', multi_class = 'multinomial', max_iter=1000)
model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/

0.3874757035409448

In [0]:
X = df[['favorites','favorites.1','favorites.2','comments','comments.1','comments.2','number','tracks','duration']]
y = df[['genre_top']].dropna().values
print(X.columns.values)
print(X.head(10).T)
print(X.shape)
print(y.shape)

['favorites' 'favorites.1' 'favorites.2' 'comments' 'comments.1'
 'comments.2' 'number' 'tracks' 'duration']
                0     1     2     3     4     5       6       7     8     9
favorites     4.0   4.0   4.0   4.0   0.0   0.0     2.0     2.0   1.0   1.0
favorites.1   9.0   9.0  74.0   9.0   0.0   0.0     5.0     5.0  11.0  11.0
favorites.2   1.0   6.0 178.0   3.0   0.0   0.0     2.0     2.0   3.0   5.0
comments      0.0   0.0   0.0   0.0   1.0   1.0     1.0     1.0   0.0   1.0
comments.1    0.0   0.0   3.0   0.0   1.0   1.0     0.0     0.0   0.0   0.0
comments.2    0.0   0.0   0.0   0.0   1.0   1.0     0.0     0.0   0.0   0.0
number        4.0   6.0   1.0   5.0   0.0   0.0     1.0     2.0   2.0   2.0
tracks        7.0   7.0   2.0   7.0   4.0   4.0     2.0     2.0   2.0   1.0
duration    237.0 206.0 161.0 207.0 837.0 509.0 1,233.0 1,231.0 296.0 253.0
(47332, 9)
(47332, 1)


In [0]:
# test 4
# mod: new dataset with training
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.5, random_state=42)

model2 = LogisticRegression(random_state=42, solver = 'lbfgs', multi_class = 'multinomial', max_iter=1000)
model2.fit(Xtrain, ytrain)
model2.score(Xtest, ytest)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/

0.3874757035409448

In [0]:
# test 4B
# mod: new dataset with NO training
model = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial', max_iter=1000)
model.fit(X,y)
model.score(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.39142651905687487

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.